In [1]:
import numpy as np
import pandas as pd
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score

In [4]:
class LoadRumoursDatasetFilterNode:
    def __init__(self, file_path_replies, file_path_posts, time_cut):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None
        self.df_final = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        post_features = ['followers','favorite_count','retweet_count','verified','rumour','id','embeddings_avg']
        
        
        self.df_replies['min_since_fst_post'] = round((self.df_replies['time'] - self.df_replies['time'].min())\
                        .dt.total_seconds() / 60,2)
        
        reply_features = ['reply_followers','reply_user_id','reply_verified','time_diff','reply_embeddings_avg',\
                          'min_since_fst_post','id','time']

        filtered_replies = self.df_replies[reply_features][(self.df_replies.time_diff <= self.time_cut)&\
                                                           (self.df_replies.min_since_fst_post <= self.time_cut)]
        
        grouped_replies = filtered_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        self.df_posts = self.df_posts[post_features]
        self.df_final = self.df_posts.merge(grouped_replies, on="id", how="inner")
        self.df_final['replies'] = self.df_final['replies'].fillna(0)
        self.df_final['first_time_diff'] = self.df_final['first_time_diff'].fillna(0)
        #self.df_final = self.df_final.drop(columns=['id'])

        
        # Initialize the Robust Scaler
        scaler = RobustScaler()
        
        # Assuming data is a DataFrame containing your dataset
        scaled_features = ['followers', 'favorite_count', 'retweet_count', 'first_time_diff']
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaler.fit_transform(self.df_final [scaled_features]),columns=scaled_features)    
        
        self.df_final [scaled_features] = scaled_data


        # One-hot encoding
        self.df_final ['verified'] = self.df_final ['verified'].astype('str').str.\
                     replace(' ', '').replace('True', '1').replace('False', '0')\
                     .astype('int64')
        
        self.df_final  = pd.concat([self.df_final , pd.get_dummies(\
                                  self.df_final ["verified"],dtype=int)], axis=1, join='inner')
        self.df_final .drop(["verified"], axis=1, inplace=True)
        self.df_final .rename(columns={1:'verified',0:'no_verified'},inplace=True)

    def get_final_dataframe(self):
        return self.df_final

In [7]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut = 1e10

processor = LoadRumoursDatasetFilterNode(file_path_replies, file_path_posts, time_cut)
processor.load_data()
processor.process_data()
df_final = processor.get_final_dataframe()


In [10]:
class LoadRumoursDataset:
    def __init__(self, file_path_replies, file_path_posts, time_cut):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None
        self.df_final = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        post_features = ['followers','favorite_count','retweet_count','verified','rumour','id','embeddings_avg']
        
        reply_features = ['reply_followers','reply_user_id','reply_verified','time_diff','reply_embeddings_avg','id','time']

        filtered_replies = self.df_replies[reply_features][self.df_replies.time_diff < self.time_cut]
        grouped_replies = filtered_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        self.df_posts = self.df_posts[post_features]
        self.df_final = self.df_posts.merge(grouped_replies, on="id", how="left")
        self.df_final['replies'] = self.df_final['replies'].fillna(0)
        self.df_final['first_time_diff'] = self.df_final['first_time_diff'].fillna(0)
        self.df_final = self.df_final.drop(columns=['id'])

        
        # Initialize the Robust Scaler
        scaler = RobustScaler()
        
        # Assuming data is a DataFrame containing your dataset
        scaled_features = ['followers', 'favorite_count', 'retweet_count', 'first_time_diff']
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaler.fit_transform(self.df_final [scaled_features]),columns=scaled_features)    
        
        self.df_final [scaled_features] = scaled_data


        # One-hot encoding
        self.df_final ['verified'] = self.df_final ['verified'].astype('str').str.\
                     replace(' ', '').replace('True', '1').replace('False', '0')\
                     .astype('int64')
        
        self.df_final  = pd.concat([self.df_final , pd.get_dummies(\
                                  self.df_final ["verified"],dtype=int)], axis=1, join='inner')
        self.df_final .drop(["verified"], axis=1, inplace=True)
        self.df_final .rename(columns={1:'verified',0:'no_verified'},inplace=True)

    def get_final_dataframe(self):
        return self.df_final

In [37]:
class HeteroDataProcessorFilterNode:
    def __init__(self, file_path_replies, file_path_posts, time_cut=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None
        self.post_map = None
        self.reply_user_map = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies['min_since_fst_post'] = round((self.df_replies['time'] - self.df_replies['time'].min())\
                        .dt.total_seconds() / 60,2)
        
        self.df_replies = self.df_replies[reply_features][(self.df_replies.time_diff <= self.time_cut)&\
                                                           (self.df_replies.min_since_fst_post <= self.time_cut)]
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Mapping post ids
        self.post_map = {value: i for i, value in enumerate(self.df_posts['id'].unique())}
        self.df_replies["id"] = self.df_replies['id'].map(self.post_map).astype(int)

        # Mapping reply user ids
        self.reply_user_map = {value: i for i, value in enumerate(self.df_replies['reply_user_id'].unique())}
        self.df_replies["reply_user_id"] = self.df_replies["reply_user_id"].map(self.reply_user_map)

    def create_features(self):
        post_features = self.df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]


        # Initialize the Robust Scaler
        scaler = RobustScaler()
        
        # Assuming data is a DataFrame containing your dataset
        scaled_features = scaler.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = post_features['no_verified']
        scaled_data['verified'] = post_features['verified']
        post_features = scaled_data

        post_embeddings = np.array(self.df_posts['embeddings_avg'].tolist())
        #post_features = self.scaler.fit_transform(post_features)
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        scaler = RobustScaler()
        reply_features = self.df_replies[["reply_followers", "reply_no_verified", "reply_verified","time_diff"]]
        reply_features[['reply_followers','time_diff']] = scaler.fit_transform(reply_features[['reply_followers','time_diff']])

        reply_embeddings = np.array(self.df_replies['reply_embeddings_avg'].tolist())
        #reply_features = self.scaler.transform(reply_features)
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        return x1, x2

    def create_heterodata(self, x1, x2):
        y = self.df_posts['rumour'].to_numpy()
        edge_index = self.df_replies[["id", "reply_user_id"]].values.transpose()

        num_rows = x1.shape[0]
        indices = np.arange(num_rows)
        np.random.shuffle(indices)
        train_end = int(0.70 * num_rows)
        val_end = train_end + int(0.15 * num_rows)
        train_indices, val_indices, test_indices = indices[:train_end], indices[train_end:val_end], indices[val_end:]

        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[train_indices], val_mask[val_indices], test_mask[test_indices] = True, True, True

        data = HeteroData()
        data['id'].x = torch.tensor(x1, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x2, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x2)))
        data = T.ToUndirected()(data)

        return data

    def process(self):
        self.load_data()
        self.process_data()
        x1, x2 = self.create_features()
        return self.create_heterodata(x1, x2)

In [ ]:
class HeteroDataProcessor:
    def __init__(self, file_path_replies, file_path_posts, time_cut=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None
        self.post_map = None
        self.reply_user_map = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies = self.df_replies[reply_features][self.df_replies.time_diff < self.time_cut]
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="left")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Mapping post ids
        self.post_map = {value: i for i, value in enumerate(self.df_posts['id'].unique())}
        self.df_replies["id"] = self.df_replies['id'].map(self.post_map).astype(int)

        # Mapping reply user ids
        self.reply_user_map = {value: i for i, value in enumerate(self.df_replies['reply_user_id'].unique())}
        self.df_replies["reply_user_id"] = self.df_replies["reply_user_id"].map(self.reply_user_map)

    def create_features(self):
        post_features = self.df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]


        # Initialize the Robust Scaler
        scaler = RobustScaler()
        
        # Assuming data is a DataFrame containing your dataset
        scaled_features = scaler.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = post_features['no_verified']
        scaled_data['verified'] = post_features['verified']
        post_features = scaled_data

        post_embeddings = np.array(self.df_posts['embeddings_avg'].tolist())
        #post_features = self.scaler.fit_transform(post_features)
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        scaler = RobustScaler()
        reply_features = self.df_replies[["reply_followers", "reply_no_verified", "reply_verified","time_diff"]]
        reply_features[['reply_followers','time_diff']] = scaler.fit_transform(reply_features[['reply_followers','time_diff']])

        reply_embeddings = np.array(self.df_replies['reply_embeddings_avg'].tolist())
        #reply_features = self.scaler.transform(reply_features)
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        return x1, x2

    def create_heterodata(self, x1, x2):
        y = self.df_posts['rumour'].to_numpy()
        edge_index = self.df_replies[["id", "reply_user_id"]].values.transpose()

        num_rows = x1.shape[0]
        indices = np.arange(num_rows)
        np.random.shuffle(indices)
        train_end = int(0.70 * num_rows)
        val_end = train_end + int(0.15 * num_rows)
        train_indices, val_indices, test_indices = indices[:train_end], indices[train_end:val_end], indices[val_end:]

        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[train_indices], val_mask[val_indices], test_mask[test_indices] = True, True, True

        data = HeteroData()
        data['id'].x = torch.tensor(x1, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x2, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x2)))
        data = T.ToUndirected()(data)

        return data

    def process(self):
        self.load_data()
        self.process_data()
        x1, x2 = self.create_features()
        return self.create_heterodata(x1, x2)

#### Tests with ML Classification Models

In [ ]:
!pwd

In [9]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut = 7

processor = LoadRumoursDatasetFilterNode(file_path_replies, file_path_posts, time_cut)
processor.load_data()
processor.process_data()
df_final = processor.get_final_dataframe()


In [5]:
df_final

NameError: name 'df_final' is not defined

In [5]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch


In [6]:
mlflow.set_experiment("GAT_test")

2024/08/09 01:40:35 INFO mlflow.tracking.fluent: Experiment with name 'GAT_test' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/rumour-detection-pheme/mlruns/3', creation_time=1723167635045, experiment_id='3', last_update_time=1723167635045, lifecycle_stage='active', name='GAT_test', tags={}>

In [47]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut =13

processor = HeteroDataProcessorFilterNode(file_path_replies, file_path_posts, time_cut)
data = processor.process()


/tmp/ipykernel_16828/3364976966.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reply_features[['reply_followers','time_diff']] = scaler.fit_transform(reply_features[['reply_followers','time_diff']])


In [48]:
data

HeteroData(
  id={
    x=[17, 106],
    y=[17],
    train_mask=[17],
    val_mask=[17],
    test_mask=[17],
  },
  reply_user_id={ x=[75, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 75] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 75] }
)

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero

class GAT(torch.nn.Module):
    def __init__(self, dim_h, dim_out):
        super().__init__()
        self.conv1 = GATConv((-1, -1), dim_h, add_self_loops=False)
        self.conv2 = GATConv(dim_h, dim_h, add_self_loops=False)  # Added second GATConv layer
        self.linear = nn.Linear(dim_h, dim_out)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index).relu()
        h = self.dropout(h)
        h = self.conv2(h, edge_index).relu()  # Pass through the second GATConv layer
        h = self.dropout(h)
        h = self.linear(h)
        return h

model = GAT(dim_h=64, dim_out=2)
model = to_hetero(model, data.metadata(), aggr='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

@torch.no_grad()
def test(mask):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
    acc = (pred[mask] == data['id'].y[mask]).sum() / mask.sum()
    return float(acc)
    
if mlflow.active_run():
    mlflow.end_run()
    
with mlflow.start_run():

    for epoch in range(500):
        model.train()
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)['id']
        mask = data['id'].train_mask
        loss = F.cross_entropy(out[mask], data['id'].y[mask])
        loss.backward()
        optimizer.step()
    
        if epoch % 50 == 0:
            train_acc = test(data['id'].train_mask)
            val_acc = test(data['id'].val_mask)
            print(f'Epoch: {epoch:>3} | Train Loss: {loss:.4f} | Train Acc: {train_acc*100:.2f}% | Val Acc: {val_acc*100:.2f}%')
    
         # Log metrics
        mlflow.log_metric("train_loss", loss.item(), step=epoch)
        mlflow.log_metric("train_acc", train_acc, step=epoch)
        mlflow.log_metric("val_acc", val_acc, step=epoch)
    
    test_acc = test(data['id'].test_mask)
    print(f'Test accuracy: {test_acc*100:.2f}%')
    
    mlflow.log_metric("test_acc", test_acc)
    
    mlflow.log_param("dim_h", 64)
    mlflow.log_param("dim_out", 2)
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("epochs", 500)
    
    mlflow.pytorch.log_model(model, "GAT_model")


Epoch:   0 | Train Loss: 0.7454 | Train Acc: 74.95% | Val Acc: 76.33%
Epoch:  50 | Train Loss: 0.4059 | Train Acc: 81.87% | Val Acc: 80.67%
Epoch: 100 | Train Loss: 0.3335 | Train Acc: 86.22% | Val Acc: 84.67%
Epoch: 150 | Train Loss: 0.2636 | Train Acc: 89.22% | Val Acc: 85.67%
Epoch: 200 | Train Loss: 0.2193 | Train Acc: 92.36% | Val Acc: 85.00%
Epoch: 250 | Train Loss: 0.1708 | Train Acc: 94.36% | Val Acc: 85.33%
Epoch: 300 | Train Loss: 0.1442 | Train Acc: 95.93% | Val Acc: 85.67%
Epoch: 350 | Train Loss: 0.1172 | Train Acc: 96.79% | Val Acc: 85.67%
Epoch: 400 | Train Loss: 0.0940 | Train Acc: 97.57% | Val Acc: 85.67%
Epoch: 450 | Train Loss: 0.0785 | Train Acc: 97.79% | Val Acc: 86.00%


2024/08/09 01:52:48 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Test accuracy: 86.05%


2024/08/09 01:52:52 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/08/09 01:52:52 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [14]:
test_mask = data['id'].test_mask
pred = model(data.x_dict, data.edge_index_dict)['id'].argmax(dim=-1)
true_labels = data['id'].y[test_mask]
pred_labels = pred[test_mask]
precision_score(true_labels, pred_labels, average='macro')

0.8090855457227139

In [15]:
recall_score(true_labels, pred_labels, average='macro')

0.8147680845950493

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut =180

df_replies = pd.read_pickle(file_path_replies)
df_posts = pd.read_pickle(file_path_posts)
post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']
# Filter and group replies
df_replies['min_since_fst_post'] = round((df_replies['time'] - df_replies['time'].min())\
.dt.total_seconds() / 60,2)

grouped_replies = df_replies.groupby(['id']).agg(
    replies=('time_diff', 'count'),
    first_time_diff=('time_diff', 'first')
).reset_index()

# Merge posts and replies
df_posts = df_posts[post_features].merge(grouped_replies, on="id", how="inner")
df_posts['replies'] = df_posts['replies'].fillna(0)
df_posts['first_time_diff'] = df_posts['first_time_diff'].fillna(0)

# One-hot encoding for verified columns
df_posts['verified'] = df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
df_posts = pd.concat([df_posts, pd.get_dummies(df_posts["verified"], dtype=int)], axis=1)
df_posts.drop(["verified"], axis=1, inplace=True)
df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

df_replies['reply_verified'] = df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
df_replies = pd.concat([df_replies, pd.get_dummies(df_replies["reply_verified"], dtype=int)], axis=1)
df_replies.drop(["reply_verified"], axis=1, inplace=True)
df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)


train, not_train = train_test_split(df_posts, test_size=0.3, random_state=42, stratify=df_posts['rumour'])

# Then, split train + validation into train and validation
val, test = train_test_split(not_train, test_size=0.5, random_state=42, stratify=not_train['rumour'])

post_features = train[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]


# Initialize the Robust Scaler
scaler_posts = RobustScaler()

# Assuming data is a DataFrame containing your dataset
scaled_features = scaler_posts.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])

# Add the binary features back to the scaled data
scaled_data['no_verified'] = np.array(train['no_verified'])
scaled_data['verified'] = np.array(train['verified'])
post_features = scaled_data

post_embeddings = np.array(train['embeddings_avg'].tolist())
#post_features = scaler.fit_transform(post_features)
x1 = np.concatenate((post_features, post_embeddings), axis=1)

scaler_replies = RobustScaler()
reply_features =  df_replies[df_replies.id.isin(np.array(train.id))][["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
reply_features[['reply_followers','time_diff']] = scaler_replies.fit_transform(reply_features[['reply_followers','time_diff']])

reply_embeddings = np.array(df_replies[df_replies.id.isin(np.array(train.id))]['reply_embeddings_avg'].tolist())
#reply_features = scaler.transform(reply_features)
x2 = np.concatenate((reply_features, reply_embeddings), axis=1)



test_val_df_replies = pd.read_pickle(file_path_replies)
test_val_df_posts = pd.read_pickle(file_path_posts)
test_val_df_posts=test_val_df_posts[~test_val_df_posts.id.isin(train.id)]
test_val_df_replies=test_val_df_replies[~test_val_df_replies.id.isin(train.id)]

post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
test_val_reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

test_val_df_replies['min_since_fst_post'] = round((test_val_df_replies['time'] - test_val_df_replies['time'].min())\
.dt.total_seconds() / 60,2)

test_val_df_replies = test_val_df_replies[test_val_reply_features][(test_val_df_replies.time_diff <= time_cut)&\
   (test_val_df_replies.min_since_fst_post <= time_cut)]
grouped_replies = test_val_df_replies.groupby(['id']).agg(
    replies=('time_diff', 'count'),
    first_time_diff=('time_diff', 'first')
).reset_index()

test_val_df_posts = test_val_df_posts[post_features].merge(grouped_replies, on="id", how="inner")
test_val_df_posts['replies'] = test_val_df_posts['replies'].fillna(0)
test_val_df_posts['first_time_diff'] = test_val_df_posts['first_time_diff'].fillna(0)

# One-hot encoding for verified columns
test_val_df_posts['verified'] = test_val_df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
test_val_df_posts = pd.concat([test_val_df_posts, pd.get_dummies(test_val_df_posts["verified"], dtype=int)], axis=1)
test_val_df_posts.drop(["verified"], axis=1, inplace=True)
test_val_df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

test_val_df_replies['reply_verified'] = test_val_df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
test_val_df_replies = pd.concat([test_val_df_replies, pd.get_dummies(test_val_df_replies["reply_verified"], dtype=int)], axis=1)
test_val_df_replies.drop(["reply_verified"], axis=1, inplace=True)
test_val_df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

test_val_df_posts = test_val_df_posts.merge(pd.concat([val,test])[['id']].reset_index(),on='id',how='left')
test_val_df_posts.set_index('index',drop=True,inplace=True)

post_features = test_val_df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]


scaled_features = scaler_posts.transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])

# Add the binary features back to the scaled data
scaled_data['no_verified'] = np.array(post_features['no_verified'])
scaled_data['verified'] = np.array(post_features['verified'])
post_features = scaled_data

post_embeddings = np.array(test_val_df_posts['embeddings_avg'].tolist())
#post_features = scaler.fit_transform(post_features)
x3 = np.concatenate((post_features, post_embeddings), axis=1)

test_val_reply_features =  test_val_df_replies[["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])

test_val_reply_embeddings = np.array(test_val_df_replies['reply_embeddings_avg'].tolist())
x4 = np.concatenate((test_val_reply_features, test_val_reply_embeddings), axis=1)


# Mapping post ids
post_map = {value: i for i, value in enumerate(pd.concat([train[['id']],test_val_df_posts[['id']]])['id'].unique())}
df_replies_edges = pd.concat([df_replies[df_replies.id.isin(np.array(train.id))][["id", "reply_user_id"]],\
                        test_val_df_replies[["id", "reply_user_id"]]])

df_replies_edges["id"] = df_replies_edges['id'].map(post_map).astype(int)

# Mapping reply user ids
reply_user_map = {value: i for i, value in enumerate(df_replies_edges['reply_user_id'].unique())}
df_replies_edges["reply_user_id"] = df_replies_edges["reply_user_id"].map(reply_user_map)


 y = pd.concat([train['rumour'],test_val_df_posts['rumour']]).to_numpy()
edge_index = df_replies_edges.values.transpose()
x = np.concatenate((x1,x3))
x_reply = np.concatenate((x2,x4))

num_rows = x.shape[0]
train_mask = np.zeros(num_rows, dtype=bool)
val_mask = np.zeros(num_rows, dtype=bool)
test_mask = np.zeros(num_rows, dtype=bool)
train_mask[:-x3.shape[0]]=True
val_mask[-x3.shape[0]:-int(x3.shape[0]/2)]=True
test_mask[-int(x3.shape[0]/2):]=True

data = HeteroData()
data['id'].x = torch.tensor(x, dtype=torch.float32)
data['id'].y =  torch.from_numpy(y)
data['id'].train_mask = torch.tensor(train_mask)
data['id'].val_mask = torch.tensor(val_mask) 
data['id'].test_mask = torch.tensor(test_mask)
data['reply_user_id'].x = torch.tensor(x_reply, dtype=torch.float32)
data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x_reply)))
data = T.ToUndirected()(data)